In [1]:
# Import Necessary Package for Experiment
import os
import openai
import time

In [1]:
# openai.organization = ""  #My OpenAI account
# openai.api_key = "" #My Key
# openai.Model.list()

### Load Dataset

In [3]:
import json
import time

In [12]:
with open('dataset/human_news_articles_2.json', 'r') as f:
    data = json.load(f)

#with open('dataset/llm_part.json', 'r') as f:
#    lllm_data = json.load(f)
print(len(data))

600


In [13]:
# Call GPT3.5 Model 
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo-16k", 
                                 temperature=0, 
                                 max_tokens=1024):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=temperature, # this is the degree of randomness of the model's output
            max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
        )
        return response.choices[0].message["content"]
    except openai.error.RateLimitError:
        return None

In [14]:
def retry_with_exponential_backoff(input_text, max_retries=5):
    retry_count = 0
    wait_time = 1  # initial wait time in seconds

    while retry_count < max_retries:
        response = get_completion_from_messages(input_text)
        if response is not None:
            return response

        time.sleep(wait_time)
        wait_time *= 2  # double the wait time for next retry
        retry_count += 1

    raise Exception("Max retries reached")

In [15]:
delimiter = "####"
step1_system_message = f"""
You are an extractive summarizer that follows the output pattern.\
Articles that need to be summarized will be delimited with {delimiter} characters.\
"""

step2_system_message = f"""
You are an abstractive summarizer that follows the output pattern.\
"""

In [16]:
i = 0

In [17]:
#for item in human_data[591:]:
for item in data:
    print(f"Summarizing {i} article")
    document = item['text']

    #if len(document) > 7500:
    #    document = document[0:7500]
    
    user_message1 = f"""
    Please extract sentences from {delimiter}{document}{delimiter} as the summary.\
    The article is newsstyle.\
    The topic is about COVID-19 or Vaccine.
    """

    messages1 =  [  
                {'role':'system', 'content': step1_system_message},    
                {'role':'user', 'content': f"{delimiter}{user_message1}{delimiter}"},  
                ]
    #extractive = get_completion_from_messages(messages1)
    extractive = retry_with_exponential_backoff(messages1)
    print(extractive)

    user_message2 = f"""
    Please revise the extracted summary {extractive} based on {document}.\
    The revised summary should include the information in the extracted summary.\
    """

    messages2 =  [  
                {'role':'system', 'content': step2_system_message},    
                {'role':'user', 'content': f"{delimiter}{user_message2}{delimiter}"},  
                ]
    #abstractive = get_completion_from_messages(messages2)
    abstractive = retry_with_exponential_backoff(messages2)
    print(abstractive)

    item['extractive_sum'] = extractive
    item['abstractive'] = abstractive
    
    time.sleep(2)
    i += 1

Summarizing 0 article
Some people may experience "long colds," or long-term symptoms following common colds, flu, pneumonia, or other respiratory illnesses, similar to the pattern seen in long COVID, according to a study from Queen Mary University of London. The study found that even people with acute respiratory infections who tested negative for COVID-19 could still experience long-term symptoms, including coughing, stomach pain, and diarrhea. Those with long COVID were more likely to report ongoing problems with sense of taste or smell, lightheadedness, or dizziness. The study analyzed questionnaire data from over 10,000 UK adults and found that long-lasting symptoms often go undiagnosed due to a wide range of symptoms and lack of testing. Understanding the long-term effects of COVID-19 and other infections is important for identifying appropriate treatment and care. Recent data from the CDC's National Center for Health Statistics shows that millions of Americans report having long 

In [18]:
len(data)

600

In [19]:
FILE_PATH = 'summary/human2_summary_by_gpt35.json'
with open(FILE_PATH, 'w') as output_file:
	print(json.dumps(data, indent=2), file=output_file)